In [ ]:
!pip install fastapi pyngrok uvicorn python-multipart PyPDF2

## Authenticate ngrok

In [ ]:
!ngrok config add-authtoken 2WXWTsDTHS7fchamHbXVlcNyvur_56bYU7JCF67SaEh3NT1ZR

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


## Extract function

In [ ]:
from PyPDF2 import PdfReader

def extract_text_with_pyPDF(PDF_File):
    pdf_reader = PdfReader(PDF_File)
    raw_text = ''
    for i, page in enumerate(pdf_reader.pages):
        text = page.extract_text()
        if text:
            raw_text += text
    return raw_text

## server

In [ ]:
from fastapi import FastAPI, UploadFile, File, HTTPException
from fastapi.middleware.cors import CORSMiddleware
import shutil
import os

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

# Define a directory where uploaded PDF files will be saved
UPLOAD_DIR = "uploads"

if not os.path.exists(UPLOAD_DIR):
    os.makedirs(UPLOAD_DIR)



# API ENDPOINT FOR UPLOADING PDF  ----------------------------------------------

@app.post('/upload-pdf/')
async def upload_pdf(file: UploadFile):
    # Check if the uploaded file is a PDF
    if not file.filename.endswith(".pdf"):
        return {"error": "Only PDF files are allowed."}

    # Define the new filename
    new_filename = "text.pdf"
    file_path = os.path.join(UPLOAD_DIR, new_filename)

    # Save the uploaded PDF file with the new name
    with open(file_path, "wb") as f:
        shutil.copyfileobj(file.file, f)

    # Extracting text
    text = extract_text_with_pyPDF("uploads/text.pdf")

    # Return a success message
    return {
        "message": "File uploaded successfully",
        "extracted_text": text
    }



# API ENDPOINT FOR PROCESSING TEXT  --------------------------------------------

@app.post('/process-text/')
async def process_text(input_data: dict):
    input_text = input_data.get('text')
    if not input_text:
        raise HTTPException(status_code=400, detail="Input text not provided")

    # Reverse the input text
    reversed_text = input_text[::-1]

    return {"reversed_text": reversed_text}



# ROOT API ENDPOINT ------------------------------------------------------------

@app.get('/')
async def root():
    return {'hello': 'world'}

import nest_asyncio
from pyngrok import ngrok
import uvicorn

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)


INFO:     Started server process [1595]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: https://5fe2-34-23-254-59.ngrok-free.app
INFO:     182.79.4.251:0 - "GET / HTTP/1.1" 200 OK
INFO:     182.66.218.125:0 - "POST /upload-pdf HTTP/1.1" 307 Temporary Redirect
INFO:     182.66.218.125:0 - "POST /upload-pdf/ HTTP/1.1" 200 OK
INFO:     182.66.218.125:0 - "POST /upload-pdf HTTP/1.1" 307 Temporary Redirect
INFO:     182.66.218.125:0 - "POST /upload-pdf/ HTTP/1.1" 200 OK
INFO:     182.66.218.125:0 - "POST /upload-pdf HTTP/1.1" 307 Temporary Redirect
INFO:     182.66.218.125:0 - "POST /upload-pdf/ HTTP/1.1" 200 OK
INFO:     182.66.218.125:0 - "POST /upload-pdf HTTP/1.1" 307 Temporary Redirect
INFO:     182.66.218.125:0 - "POST /upload-pdf/ HTTP/1.1" 200 OK
INFO:     182.66.218.125:0 - "POST /upload-pdf HTTP/1.1" 307 Temporary Redirect
INFO:     182.66.218.125:0 - "POST /upload-pdf/ HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [1595]
